In [ ]:
#load tools
from tools import GetDOIsTool, GetDOIsCleanTool, GetOriginaltextTool, \
    GetExtracttextTool, StreamModeTool, CleanDBTool, FileRagTool, QueryTool,\
    DataextTool, DatacountTool, FeaextTool, ModelSelTool, ShapplotTool

tools = [
    GetDOIsTool(description="Quickly get the doi of the literature."),
    GetDOIsCleanTool(description="Quickly clean the doi of the literature."),
    GetOriginaltextTool(description="Get the original text of the literature."),
    GetExtracttextTool(description="Get the extracted text of the literature."),
    StreamModeTool(description="create the database."),
    CleanDBTool(description="Clean the database."),
    FileRagTool(description="File quiz tool. analysis file"),
    QueryTool(description="File quiz tool. Use when you need to quiz based on uploaded files, no need to upload files again for subsequent queries."),
    DataextTool(description="Extract the jsonl to csv."),
    DatacountTool(description="Count the data."),
    FeaextTool(description="Extract the feature."),
    ModelSelTool(description="Select the model."),
    ShapplotTool(description="Plot the shap.")
]

# print(f"construct {len(tools)} tools:")
# for tool in tools:
#     print(f"- {tool.name}: {tool.description}")
#     if tool.args_schema:
#         print(f" params: {tool.args_schema.model_json_schema()}")

In [ ]:
#define the agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
import warnings
warnings.filterwarnings("ignore")

memory = MemorySaver()

model = ChatOpenAI(
    openai_api_key="", # your api key
    base_url="", # your api url
    model= "qwen-plus"
)
with open("prompts/calling_prompts_gradio.txt", "r") as f:
    prompts = f.read()

In [ ]:
agent_executor = create_react_agent(model, tools, prompt=prompts, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

while True:
    user_input = input("please enter your problem (type 'exit' to end the conversation): ")
    if user_input.strip().lower() == "exit":
        break

    for chunk in agent_executor.stream(
        {"messages": [HumanMessage(content = user_input)]}, config
    ):
        if "agent" in chunk.keys():
            
            if chunk['agent']['messages'][0].content:
               
                print(f"Agent: {chunk['agent']['messages'][0].content}")
                print("----")
        # print(chunk)
